In [65]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


from collections import defaultdict
from scipy.stats.stats import pearsonr
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import pdist, squareform
from scipy.stats import mode
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram

In [66]:
df = pd.read_csv('training.csv')

In [67]:
df['MMRAcquisitionAuctionAveragePrice'].fillna(df['MMRAcquisitionAuctionAveragePrice'].median(), inplace = True)
df['MMRAcquisitionAuctionCleanPrice'].fillna(df['MMRAcquisitionAuctionCleanPrice'].median(), inplace = True)
df['MMRAcquisitionRetailAveragePrice'].fillna(df['MMRAcquisitionRetailAveragePrice'].median(), inplace = True)
df['MMRAcquisitonRetailCleanPrice'].fillna(df['MMRAcquisitonRetailCleanPrice'].median(), inplace = True)
df['MMRCurrentAuctionAveragePrice'].fillna(df['MMRCurrentAuctionAveragePrice'].median(), inplace = True)
df['MMRCurrentAuctionCleanPrice'].fillna(df['MMRCurrentAuctionCleanPrice'].median(), inplace = True)
df['MMRCurrentRetailAveragePrice'].fillna(df['MMRCurrentRetailAveragePrice'].median(), inplace = True)
df['MMRCurrentRetailCleanPrice'].fillna(df['MMRCurrentRetailCleanPrice'].median(), inplace = True)

In [68]:
df['MMR_factor'] = ( df['MMRAcquisitionAuctionAveragePrice'] + df['MMRAcquisitonRetailCleanPrice'] + df['MMRCurrentAuctionCleanPrice'] + df['MMRCurrentRetailAveragePrice']) / 4

In [69]:
#BINNING NUMERICAL VALUES
#We have to use the sturges' rule but that gives us 17 bin but i think 10 is better
df['VehBCost_bin'] = pd.cut(df.VehBCost.astype(int), 10, right = False) 
df['VehOdo_bin'] = pd.cut(df.VehOdo.astype(int), 10, right = False)
df['MMR_factor_bin'] = pd.cut(df.MMR_factor.astype(int), 10, right = False)
df['WarrantyCost_bin'] = pd.cut(df.WarrantyCost.astype(int), 10, right = False)

In [70]:
#SELECT THE COLUMNS I AM INTERESTED IN
df_new = df[['Auction','VehicleAge','Make','Trim','Color','Transmission','WheelType','Nationality','Size',
            'TopThreeAmericanName','VNZIP1','VNST','VehBCost_bin', 'VehOdo_bin','WarrantyCost_bin']]

In [71]:
#FIX MISSING VALUES
df_new.isnull().sum()

Auction                    0
VehicleAge                 0
Make                       0
Trim                    1911
Color                      7
Transmission               8
WheelType               2577
Nationality                4
Size                       4
TopThreeAmericanName       4
VNZIP1                     0
VNST                       0
VehBCost_bin               0
VehOdo_bin                 0
WarrantyCost_bin           0
dtype: int64

In [72]:
df_new.dropna(subset = ['Transmission','Color','Nationality','Size','TopThreeAmericanName'], inplace=True)

C:\Users\bonin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [73]:
df_new.Trim.fillna('Bas', inplace = True, limit=801)

C:\Users\bonin\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [74]:
df_new.Trim.fillna('LS', inplace = True, limit=580)

In [75]:
df_new.Trim.fillna('SE', inplace = True, limit=530)

In [76]:
df_new.WheelType.fillna('Alloy', inplace = True, limit=1377)

In [77]:
df_new.WheelType.fillna('Covers', inplace = True, limit=1200)

In [78]:
df_new.Color.isnull().sum()
df_new.Transmission.isnull().sum()
df_new.Nationality.isnull().sum()
df_new.Size.isnull().sum()
df_new.TopThreeAmericanName.isnull().sum()
df_new.Trim.isnull().sum()
df_new.WheelType.isnull().sum()

0

In [79]:
df_new.isnull().sum()

Auction                 0
VehicleAge              0
Make                    0
Trim                    0
Color                   0
Transmission            0
WheelType               0
Nationality             0
Size                    0
TopThreeAmericanName    0
VNZIP1                  0
VNST                    0
VehBCost_bin            0
VehOdo_bin              0
WarrantyCost_bin        0
dtype: int64

In [80]:
df_new.head()

,Auction,VehicleAge,Make,Trim,Color,Transmission,WheelType,Nationality,Size,TopThreeAmericanName,VNZIP1,VNST,VehBCost_bin,VehOdo_bin,WarrantyCost_bin
0,ADESA,2,KIA,EX,BLUE,AUTO,Covers,OTHER ASIAN,MEDIUM,OTHER,33619,FL,"[3649.4, 7297.8)","[60271.0, 71360.2)","[462.0, 1165.6)"
1,MANHEIM,3,DODGE,SXT,SILVER,AUTO,Covers,AMERICAN,MEDIUM,CHRYSLER,32124,FL,"[3649.4, 7297.8)","[82449.4, 93538.6)","[1165.6, 1869.2)"
2,OTHER,2,DODGE,SE,BLACK,AUTO,Covers,AMERICAN,MEDIUM,CHRYSLER,74135,OK,"[7297.8, 10946.2)","[38092.6, 49181.8)","[462.0, 1165.6)"
3,OTHER,4,FORD,S,BROWN,AUTO,Covers,AMERICAN,VAN,FORD,85018,AZ,"[3649.4, 7297.8)","[71360.2, 82449.4)","[1165.6, 1869.2)"
4,MANHEIM,4,CHRYSLER,Bas,BLUE,AUTO,Alloy,AMERICAN,VAN,CHRYSLER,75236,TX,"[3649.4, 7297.8)","[71360.2, 82449.4)","[1165.6, 1869.2)"


In [81]:
#this code is necessary because if we don't add a string to the algorithm value..
#..when we run the algorithm it doesn't find the attribute values
for i in df_new:
        df_new[i] = df_new[i].astype(str) + "_{}".format(i)

C:\Users\bonin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [82]:
df_new.head()

,Auction,VehicleAge,Make,Trim,Color,Transmission,WheelType,Nationality,Size,TopThreeAmericanName,VNZIP1,VNST,VehBCost_bin,VehOdo_bin,WarrantyCost_bin
0,ADESA_Auction,2_VehicleAge,KIA_Make,EX_Trim,BLUE_Color,AUTO_Transmission,Covers_WheelType,OTHER ASIAN_Nationality,MEDIUM_Size,OTHER_TopThreeAmericanName,33619_VNZIP1,FL_VNST,"[3649.4, 7297.8)_VehBCost_bin","[60271.0, 71360.2)_VehOdo_bin","[462.0, 1165.6)_WarrantyCost_bin"
1,MANHEIM_Auction,3_VehicleAge,DODGE_Make,SXT_Trim,SILVER_Color,AUTO_Transmission,Covers_WheelType,AMERICAN_Nationality,MEDIUM_Size,CHRYSLER_TopThreeAmericanName,32124_VNZIP1,FL_VNST,"[3649.4, 7297.8)_VehBCost_bin","[82449.4, 93538.6)_VehOdo_bin","[1165.6, 1869.2)_WarrantyCost_bin"
2,OTHER_Auction,2_VehicleAge,DODGE_Make,SE_Trim,BLACK_Color,AUTO_Transmission,Covers_WheelType,AMERICAN_Nationality,MEDIUM_Size,CHRYSLER_TopThreeAmericanName,74135_VNZIP1,OK_VNST,"[7297.8, 10946.2)_VehBCost_bin","[38092.6, 49181.8)_VehOdo_bin","[462.0, 1165.6)_WarrantyCost_bin"
3,OTHER_Auction,4_VehicleAge,FORD_Make,S_Trim,BROWN_Color,AUTO_Transmission,Covers_WheelType,AMERICAN_Nationality,VAN_Size,FORD_TopThreeAmericanName,85018_VNZIP1,AZ_VNST,"[3649.4, 7297.8)_VehBCost_bin","[71360.2, 82449.4)_VehOdo_bin","[1165.6, 1869.2)_WarrantyCost_bin"
4,MANHEIM_Auction,4_VehicleAge,CHRYSLER_Make,Bas_Trim,BLUE_Color,AUTO_Transmission,Alloy_WheelType,AMERICAN_Nationality,VAN_Size,CHRYSLER_TopThreeAmericanName,75236_VNZIP1,TX_VNST,"[3649.4, 7297.8)_VehBCost_bin","[71360.2, 82449.4)_VehOdo_bin","[1165.6, 1869.2)_WarrantyCost_bin"


In [83]:
#we have to pass the values of df_new as a list to run in the right way apriori algorithm
baskets = df_new.values.tolist()

In [84]:
#BASKET EXAMPLE
baskets[0]

['ADESA_Auction',
 '2_VehicleAge',
 'KIA_Make',
 'EX_Trim',
 'BLUE_Color',
 'AUTO_Transmission',
 'Covers_WheelType',
 'OTHER ASIAN_Nationality',
 'MEDIUM_Size',
 'OTHER_TopThreeAmericanName',
 '33619_VNZIP1',
 'FL_VNST',
 '[3649.4, 7297.8)_VehBCost_bin',
 '[60271.0, 71360.2)_VehOdo_bin',
 '[462.0, 1165.6)_WarrantyCost_bin']

In [85]:
from fim import apriori

In [86]:
#This is the explaination of the parameters of apriori
print(apriori.__doc__)

apriori (tracts, target='s', supp=10, zmin=1, zmax=None, report='a',
         eval='x', agg='x', thresh=10, prune=None, algo='b', mode='',
         border=None)
Find frequent item sets with the Apriori algorithm.
tracts  transaction database to mine (mandatory)
        The database must be an iterable of transactions;
        each transaction must be an iterable of items;
        each item must be a hashable object.
        If the database is a dictionary, the transactions are
        the keys, the values their (integer) multiplicities.
target  type of frequent item sets to find     (default: s)
        s/a   sets/all   all     frequent item sets
        c     closed     closed  frequent item sets
        m     maximal    maximal frequent item sets
        g     gens       generators
        r     rules      association rules
supp    minimum support of an item set         (default: 10)
        (positive: percentage, negative: absolute number)
conf    minimum confidence of an assoc. rul

In [92]:
rules = apriori(baskets, supp=20, zmin=2, target='r', conf=80, report='YXlC') 
print(len(rules))
contare = 0
for y in rules:
        print('%s -- %s' % (str(contare), y))
        contare = contare + 1

125
0 -- ('AMERICAN_Nationality', ('SILVER_Color',), 83.58515777572207, 20.233323054784663, 0.9922770352181137, 82.93963254593176)
1 -- ('AUTO_Transmission', ('SILVER_Color',), 96.49158872100593, 20.233323054784663, 1.0005596795627067, 96.54559309118618)
2 -- ('AMERICAN_Nationality', ('Bas_Trim',), 83.58515777572207, 20.57594134374893, 0.9695785708485045, 81.04237782033137)
3 -- ('AUTO_Transmission', ('Bas_Trim',), 96.49158872100593, 20.57594134374893, 0.9892485788534935, 95.4541670135709)
4 -- ('AMERICAN_Nationality', ('[82449.4, 93538.6)_VehOdo_bin', 'AUTO_Transmission'), 83.58515777572207, 20.27957652379484, 1.0436783505509324, 87.23601959790506)
5 -- ('AMERICAN_Nationality', ('[82449.4, 93538.6)_VehOdo_bin',), 83.58515777572207, 20.86374070647891, 1.03665713485919, 86.64915017653337)
6 -- ('AUTO_Transmission', ('[82449.4, 93538.6)_VehOdo_bin',), 96.49158872100593, 20.86374070647891, 1.0073427105786354, 97.200098530257)
7 -- ('AMERICAN_Nationality', ('3_VehicleAge', 'AUTO_Transmissi

In [93]:
#RULES EXAMPLE
#INDEX: 
# 0 : 'AMERICAN_Nationality'
# 1 : ('[82449.4, 93538.6)_VehOdo_bin', 'AUTO_Transmission')
# 2 : 83.58515777572207 refers to 'Y' in report='YXlC'  that is relative head(head is 'AMERICAN_Nationality') item support as a percentage
# 3 : 20.27957652379484 refers to 'X' in report='YXlC'  that is X relative body(body is ('[82449.4, 93538.6)_VehOdo_bin', 'AUTO_Transmission')) set  support as a percentage
# 4 : 1.0436783505509324 refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
# 5 : 87.23601959790506 refers to 'C' in report='YXlC'  that is rule confidence as a percentage
rules[4]

('AMERICAN_Nationality',
 ('[82449.4, 93538.6)_VehOdo_bin', 'AUTO_Transmission'),
 83.58515777572207,
 20.27957652379484,
 1.0436783505509324,
 87.23601959790506)

In [94]:
#Code that filters all the association rules with a certain lift under
count = 0
for r in rules:
    if r[4]>1.4: #r[4] indeed refers to index 4 so as we have seen before index 4 refers to refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
        print('%s -- %s' % (str(count), r))
        count = count + 1
print('Rules with high lift: {}'.format(count))

0 -- ('GM_TopThreeAmericanName', ('CHEVROLET_Make', 'AMERICAN_Nationality', 'AUTO_Transmission'), 34.681536300407714, 23.244937814780553, 2.883378612002964, 100.0)
1 -- ('GM_TopThreeAmericanName', ('CHEVROLET_Make', 'AMERICAN_Nationality'), 34.681536300407714, 23.70918559632713, 2.883378612002964, 100.0)
2 -- ('GM_TopThreeAmericanName', ('CHEVROLET_Make', 'AUTO_Transmission'), 34.681536300407714, 23.244937814780553, 2.883378612002964, 100.0)
3 -- ('GM_TopThreeAmericanName', ('CHEVROLET_Make',), 34.681536300407714, 23.70918559632713, 2.883378612002964, 100.0)
Rules with high lift: 4


In [96]:
#TASKS ON KAGGLE:
# 1) Frequent patterns extraction with different values of support and different types (i.e. frequent, close, maximal), (6 points)
# 2) Discussion of the most interesting frequent patterns and analyze how changes the number of patterns w.r.t. the min_sup parameter (7 points)

In [97]:
# s/a   sets/all   all     frequent item sets
for i in range(5,101,5):
    itemsets = apriori(baskets, supp=i, zmin=2, target='a') 
    print("Min support = {} \n Itemsets = {}".format(i,len(itemsets)))

Min support = 5 
 Itemsets = 2765
Min support = 10 
 Itemsets = 595
Min support = 15 
 Itemsets = 219
Min support = 20 
 Itemsets = 104
Min support = 25 
 Itemsets = 59
Min support = 30 
 Itemsets = 39
Min support = 35 
 Itemsets = 17
Min support = 40 
 Itemsets = 13
Min support = 45 
 Itemsets = 10
Min support = 50 
 Itemsets = 3
Min support = 55 
 Itemsets = 2
Min support = 60 
 Itemsets = 1
Min support = 65 
 Itemsets = 1
Min support = 70 
 Itemsets = 1
Min support = 75 
 Itemsets = 1
Min support = 80 
 Itemsets = 1
Min support = 85 
 Itemsets = 0
Min support = 90 
 Itemsets = 0
Min support = 95 
 Itemsets = 0
Min support = 100 
 Itemsets = 0


In [98]:
# c     closed     closed  frequent item sets
for i in range(5,101,5):
    itemsets = apriori(baskets, supp=i, zmin=2, target='c')
    print("Min support = {} \n Itemsets = {}".format(i,len(itemsets)))

Min support = 5 
 Itemsets = 1988
Min support = 10 
 Itemsets = 477
Min support = 15 
 Itemsets = 187
Min support = 20 
 Itemsets = 95
Min support = 25 
 Itemsets = 57
Min support = 30 
 Itemsets = 37
Min support = 35 
 Itemsets = 17
Min support = 40 
 Itemsets = 13
Min support = 45 
 Itemsets = 10
Min support = 50 
 Itemsets = 3
Min support = 55 
 Itemsets = 2
Min support = 60 
 Itemsets = 1
Min support = 65 
 Itemsets = 1
Min support = 70 
 Itemsets = 1
Min support = 75 
 Itemsets = 1
Min support = 80 
 Itemsets = 1
Min support = 85 
 Itemsets = 0
Min support = 90 
 Itemsets = 0
Min support = 95 
 Itemsets = 0
Min support = 100 
 Itemsets = 0


In [99]:
#m     maximal    maximal frequent item sets
for i in range(5,101,5):
    itemsets = apriori(baskets, supp=i, zmin=2, target='m') 
    print("Min support = {} \n Itemsets = {}".format(i,len(itemsets)))

Min support = 5 
 Itemsets = 461
Min support = 10 
 Itemsets = 129
Min support = 15 
 Itemsets = 61
Min support = 20 
 Itemsets = 30
Min support = 25 
 Itemsets = 20
Min support = 30 
 Itemsets = 16
Min support = 35 
 Itemsets = 8
Min support = 40 
 Itemsets = 6
Min support = 45 
 Itemsets = 5
Min support = 50 
 Itemsets = 3
Min support = 55 
 Itemsets = 2
Min support = 60 
 Itemsets = 1
Min support = 65 
 Itemsets = 1
Min support = 70 
 Itemsets = 1
Min support = 75 
 Itemsets = 1
Min support = 80 
 Itemsets = 1
Min support = 85 
 Itemsets = 0
Min support = 90 
 Itemsets = 0
Min support = 95 
 Itemsets = 0
Min support = 100 
 Itemsets = 0


In [100]:
#TASK ON KAGGLE:
# 3) Association rules extraction with different values of confidence (6 points)
# 4) Discussion of the most interesting rules and analyze how changes the number of rules w.r.t. the min_conf parameter, histogram of rules' confidence and lift (7 points)

In [101]:
#EXTRACT RULES
for i in range(5,101,5):
    for j in range(5,101,5):
        rules = apriori(baskets, supp=i, zmin=2, target='r', conf=j, report='YXlC') 
        print("Support = {} \nConfidence = {} \nRules = {}".format(i,j,len(rules)))
        #Code that filters all the association rules with a certain lift under
        count = 0
        for r in rules:
            if r[4]>1.4: #r[4] indeed refers to index 4 so as we have seen before index 4 refers to refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
                count = count + 1
        print('Rules with high lift: {}'.format(count))
        print('\n')

Support = 5 
Confidence = 5 
Rules = 129220
Rules with high lift: 32205


Support = 5 
Confidence = 10 
Rules = 89223
Rules with high lift: 20197


Support = 5 
Confidence = 15 
Rules = 66354
Rules with high lift: 16065


Support = 5 
Confidence = 20 
Rules = 47679
Rules with high lift: 12485


Support = 5 
Confidence = 25 
Rules = 33216
Rules with high lift: 9459


Support = 5 
Confidence = 30 
Rules = 25544
Rules with high lift: 7157


Support = 5 
Confidence = 35 
Rules = 20409
Rules with high lift: 5196


Support = 5 
Confidence = 40 
Rules = 17038
Rules with high lift: 4336


Support = 5 
Confidence = 45 
Rules = 14214
Rules with high lift: 3767


Support = 5 
Confidence = 50 
Rules = 11888
Rules with high lift: 3208


Support = 5 
Confidence = 55 
Rules = 9620
Rules with high lift: 2614


Support = 5 
Confidence = 60 
Rules = 6860
Rules with high lift: 2152


Support = 5 
Confidence = 65 
Rules = 5545
Rules with high lift: 1776


Support = 5 
Confidence = 70 
Rules = 4610
Rules w

In [117]:
#HERE WE CAN SEE THE RULES THAT WE WANT
rules = apriori(baskets, supp=20, zmin=2, target='r', conf=70, 
                report='YXlC') 
print('Number of rules:', len(rules))
#Code that filters all the association rules with a certain lift under
c = 0
for r in rules:
        if r[4]>1.4: #r[4] indeed refers to index 4 so as we have seen before index 4 refers to refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
            c += 1
            print('%s -- %s' % (str(c), r))
print('Rules with high lift: {}'.format(c))

Number of rules: 157
1 -- ('GM_TopThreeAmericanName', ('CHEVROLET_Make', 'AMERICAN_Nationality', 'AUTO_Transmission'), 34.681536300407714, 23.244937814780553, 2.883378612002964, 100.0)
2 -- ('GM_TopThreeAmericanName', ('CHEVROLET_Make', 'AMERICAN_Nationality'), 34.681536300407714, 23.70918559632713, 2.883378612002964, 100.0)
3 -- ('GM_TopThreeAmericanName', ('CHEVROLET_Make', 'AUTO_Transmission'), 34.681536300407714, 23.244937814780553, 2.883378612002964, 100.0)
4 -- ('GM_TopThreeAmericanName', ('CHEVROLET_Make',), 34.681536300407714, 23.70918559632713, 2.883378612002964, 100.0)
5 -- ('[462.0, 1165.6)_WarrantyCost_bin', ('MEDIUM_Size', 'Covers_WheelType', 'AUTO_Transmission'), 50.06509747490321, 24.20940829821496, 1.586810362131263, 79.44381545428814)
6 -- ('MEDIUM_Size', ('Covers_WheelType', '[462.0, 1165.6)_WarrantyCost_bin', 'AUTO_Transmission'), 42.21228629184226, 25.994449583718783, 1.7527693406859874, 73.9884012126005)
7 -- ('[462.0, 1165.6)_WarrantyCost_bin', ('MEDIUM_Size', 'Co

In [63]:
df_new.isnull().sum()

Auction                 0
VehicleAge              0
Make                    0
Trim                    0
Color                   0
Transmission            0
WheelType               0
Nationality             0
Size                    0
TopThreeAmericanName    0
VNZIP1                  0
VNST                    0
VehBCost_bin            0
VehOdo_bin              0
MMR_factor_bin          0
WarrantyCost_bin        0
dtype: int64

In [64]:
df.isnull().sum()

RefId                                    0
IsBadBuy                                 0
PurchDate                                0
Auction                                  0
VehYear                                  0
VehicleAge                               0
Make                                     0
Model                                    0
Trim                                  1911
SubModel                                 7
Color                                    7
Transmission                             8
WheelTypeID                           2573
WheelType                             2577
VehOdo                                   0
Nationality                              4
Size                                     4
TopThreeAmericanName                     4
MMRAcquisitionAuctionAveragePrice        0
MMRAcquisitionAuctionCleanPrice          0
MMRAcquisitionRetailAveragePrice         0
MMRAcquisitonRetailCleanPrice            0
MMRCurrentAuctionAveragePrice            0
MMRCurrentA

In [103]:
df_new.columns

Index(['Auction', 'VehicleAge', 'Make', 'Trim', 'Color', 'Transmission',
       'WheelType', 'Nationality', 'Size', 'TopThreeAmericanName', 'VNZIP1',
       'VNST', 'VehBCost_bin', 'VehOdo_bin', 'WarrantyCost_bin'],
      dtype='object')

In [114]:
# count how many rows for each attribute
for i in df_new:
    conta = 0
    for j in df_new.iterrows():
        conta += 1
    print(i, " = ", conta)

Auction  =  58374
VehicleAge  =  58374
Make  =  58374
Trim  =  58374
Color  =  58374
Transmission  =  58374
WheelType  =  58374
Nationality  =  58374
Size  =  58374
TopThreeAmericanName  =  58374
VNZIP1  =  58374
VNST  =  58374
VehBCost_bin  =  58374
VehOdo_bin  =  58374
WarrantyCost_bin  =  58374
